# 文本摘要简介

## 1、关键词提取  

### 1.1 TF-IDF 算法关键词提取  

- 输入文本进行分词，统计每个词语在该文本中的词频数（可以限制使用特定词性的词语）
- 在较大语料上预先统计出全量(常用词)词的 idf 值，得到输入文本中每个词语的 tf_idf 值
- 将 tf_idf 值作为权重，对统计得到的词语进行排序

  TF=（词语在文章中出现次数）/ （文章总词数）  
  IDF=log (语料库文档总数/(包含该词的文档数+1))  
  TF-IDF = TF * IDF  
    
- 选取前 K 个词语作为关键词，就完成了关键词的抽取


TF-IDF 方法虽然简单，但经典有效，速度也快，具有较强的普适性，该算法基本能应付大部分关键词抽取的场景。有些场景中，可以提高文本首段和标题中的词的权重。分词和词性标注的性能对关键词抽取的效果影响很大。


下面是 jieba 抽取关键词的代码。   

```
def extract_tags(self, sentence, topK=20, withWeight=False, allowPOS=(), withFlag=False):
    """
    Extract keywords from sentence using TF-IDF algorithm.
    Parameter:
        - topK: return how many top keywords. `None` for all possible words.
        - withWeight: if True, return a list of (word, weight);
                      if False, return a list of words.
        - allowPOS: the allowed POS list eg. ['ns', 'n', 'vn', 'v','nr'].
                    if the POS of w is not in this list,it will be filtered.
        - withFlag: only work with allowPOS is not empty.
                    if True, return a list of pair(word, weight) like posseg.cut
                    if False, return a list of words
    """
    if allowPOS:
        allowPOS = frozenset(allowPOS)
        words = self.postokenizer.cut(sentence)
    else:
        words = self.tokenizer.cut(sentence)
    freq = {}
    for w in words:
        if allowPOS:
            if w.flag not in allowPOS:
                continue
            elif not withFlag:
                w = w.word
        wc = w.word if allowPOS and withFlag else w
        if len(wc.strip()) < 2 or wc.lower() in self.stop_words:
            continue
        freq[w] = freq.get(w, 0.0) + 1.0
    total = sum(freq.values())
    for k in freq:
        kw = k.word if allowPOS and withFlag else k
        freq[k] *= self.idf_freq.get(kw, self.median_idf) / total

    if withWeight:
        tags = sorted(freq.items(), key=itemgetter(1), reverse=True)
    else:
        tags = sorted(freq, key=freq.__getitem__, reverse=True)
    if topK:
        return tags[:topK]
    else:
        return tags
```

### 1.2 TextRank 算法关键词提取  

- 输入文本进行分词
- 以固定窗口大小(默认为5，通过span属性调整)，统计词之间的共现关系，共现一次就加 1，构建图
- 计算图中节点的 PageRank
- 排序，权重最大的前 K 个节点作为关键词输出

TextRank 从图模型的角度提取文本的关键词，由于涉及网络构建和迭代计算，效率较低，实际应用效果并不比TFIDF有明显优势。好处是不用预先在较大的语料上计算 idf 值。  

- pagerank 简介

基本思想来源于 pagerank 算法，pagerank 算法有两个基本思想，如果一个网页被很多其他的网页链接到，就说明该网页很重要；如果一个网页被一个权值很高的网页链接到，则也要增加该网页的权重，pagerank 迭代计算公式如下所示：

$S(V_i) = (1-d) + d * \sum_{j \in In(V_i)} \frac {1}{|Out(V_j)|} S(V_j)$  

该公式中，$V_i$ 表示某个网页，$V_j$ 表示链接到 $V_i$ 的网页（即Vi的入链），$S(V_i)$ 表示网页 $V_i$ 的 $PR$ 值，$In(V_i)$ 表示网页 $V_i$ 的所有入链的集合，$Out(V_j)$ 是网页 $V_j$ 链接指向的网页的集合。$|Out(V_j)|$ 是集合中元素的个数。$d$ 表示阻尼系数，如果仅仅有求和的部分，而没有阻尼系数部分，那么该公式将无法处理没有入链的网页的 $PR$ 值，因为根据该公式这些网页的 $PR$ 值为 $0$，但实际情况却不是这样，所以加入了一个阻尼系数来确保每个网页都有一个大于 $0$ 的 $PR$值，根据实验的结果，在 $0.85$ 的阻尼系数下，大约 $100$ 多次迭代 $PR$ 值能收敛到一个稳定的值，而当阻尼系数接近 $1$ 时，需要的迭代次数会陡然增加很多，且排序不稳定。公式中 $S(V_j)$ 前面的分数指的是 $V_j$ 所有出链指向的网页应该平分 $Vj$ 的 $PR$ 值，这样才算是把自己的票分给了自己链接到的网页。  

- textrank 简介  

textrank 迭代计算公式如下所示：

$WS(V_i) = (1-d) + d * \sum_{j \in In(V_i)} \frac {w_{i,j}}{\sum_{v_k \in Out(v_j)}w_{j,k}} WS(V_j)$  

可以看到该公式比 pagerank 多了一个权重项，这里使用两个节点之间的共现次数作为权重  

- jieba textrank 排序过程  

1、使用文本中词语个数的倒数作为每个词语的初始权重，计算每个词语的 $\sum_{v_k \in Out(v_j)}w_{j,k}$

```
wsdef = 1.0 / (len(self.graph) or 1.0)
for n, out in self.graph.items():
    ws[n] = wsdef
    outSum[n] = sum((e[2] for e in out), 0.0)

```
2、迭代计算 $WS(V_i)$  

```
sorted_keys = sorted(self.graph.keys())
for x in xrange(10):  # 10 iters
    for n in sorted_keys:                          # n 就是公式中的 V_i
        s = 0                                      # s 是公式中的 \sum In 部分
        for e in self.graph[n]:                    # 遍历每个与 V_i 共现的词语 V_j
            s += e[2] / outSum[e[1]] * ws[e[1]]    # 计算出 V_j 权重对 V_i 权重的攻陷程度
        ws[n] = (1 - self.d) + self.d * s          # 更新 V_j 的权重
```

3、对词语权重进行修正，具体原因不详、可能是为了平滑的作用  

4、按照词语权重从大到小排列词语，输出前 K 个作为关键词  


### 1.3 TextRank 关键短语提取  

提取关键词短语的方法基于关键词提取，可以简单认为：如果提取出的若干关键词在文本中相邻，那么构成一个被提取的关键短语。

## 2、摘要提取  

### 2.1 TextRank 摘要提取  

与提取关键词的算法一样，只是这里将文本中每个句子看成图中的一个节点，计算两个句子之间有相似度，作为无向加权图中的权重。通过 textrank 公式计算得到权重最高的若干句子作为摘要。 

句子相似度使用公式 $S(S_i,S_j) = \frac {\{w_k|w_k \in S_i \ and \ w_k \in S_j \}}{\log(|S_i|) + \log(|S_j|)}$ 计算，其中分子是在两个句子中都出现的单词的数量，$|S_i|$是句子 $i$ 的单词数。  

句子相似度理论上也可以使用其他方式计算，如余弦距离，DNN等方法。  

### 2.2 Feature-Based 摘要提取  

#### 2.2.1 TextTeaser 算法简介  

算法主要考虑以下几个方面：

1）句子长度，长度为某个长度的句子为最理想的长度，依照距离这个长度的远近来打分。

2）句子位置，根据句子在全文中的位置，给出分数。（比如每段的第一句是核心句的比例大概是70%）

3）句子关键词打分，文本进行预处理之后，按照词频统计出排名前10的关键词，通过比较句子中包含关键词的情况，以及关键词分布的情况来打分。 

综合上述 3 步的打分做累加，然后倒排得到每个句子的重要性得分，此时要考虑到摘要的可读性，通俗的做法是按照句子在文章中出现的顺序来输出。  

计算过程如下所示：  

1、对输入文本分句  

2、对文本分词，统计文本中词频数前十的词(过滤停用词)，并且给没个词计算一个权重，作为文本关键词  

3、对文本标题分词，得到标题中的词汇(过滤停用词)  

4、遍历所有的句子并分词，计算句子与标题词汇的相似度得分，句子长度得分，句子与文本关键词的相似度得分，句子位置得分(越靠前得分越高)  

5、将上述计算的各个得分合并起来，计算句子的最终得分，并排序，去前 K 个句子作为文本的摘要  

#### 2.2.2 SummaRuNNer 算法简介  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/summary/SummaRuNNer.jpg?raw=true)
<center> **SummaRuNNer 模型** </center >

模型计算过程如下所示：

1、模型由一个两层的 bi-directional GRU-RNN 组成，第一层 RNN 操作在单词级，计算每个单词的隐藏层状态表示  

2、第二层RNN操作在句子级，输入为 word-level 层的隐藏层向量经平均池化（average pooling）、首尾拼接而成的向量，得到的隐藏层向量作为文档中句子的表示  

3、sentence-level 层隐藏层向量同样先经过平均池化、首尾拼接，然后再经过一个非线性变换，最终的结果作为整个文档的表示  

4、将文档中句子的表示与文档的表示结合起来，计算当前句子和文章表示的相似度，计算出每个句子是摘要的概率，在最终选取摘要的时候并不是简单的分类，而是根据每个句子的概率高低排序，选择概率最高的前几句即可。  


### 2.3 Abstractive 摘要  

生成式文本摘要以一种更接近于人的方式，在保留文本意图的情况下生成摘要。  

优点：可以使用不在原始输入中的词语，理论上可以生成更加通顺和自然的摘要  
缺点：让模型生成一些短语并且使句子通顺还是有难度的，通常情况下生成的效果没有抽取式效果好

文本摘要问题是根据一篇长文来生成一篇短摘要或者标题，可以看做将输入文本序列映射为一个目标文本序列，也就是 seq2seq 的问题。目前，解决seq2seq 的问题，都是借鉴 machine translation 的方法。

- encoder-decoder

编码器负责将输入的原文本编码成一个向量，该向量是原文本的一个表征，包含了文本背景。解码器负责从这个向量提取重要信息，生成文本摘要。seq2seq 架构中的编码器和解码器通常由递归神经网络（RNN）或卷积神经网络（CNN）实现，通常在解码过程中会加入 attention 模块，提升摘要效果。  


### 2.4 Combination Approach 摘要  

联合使用抽取式和生成式摘要的方法进行摘要  

#### 2.4.1 Pointer-Generator Network  

通过调整概率来组合使用 extractive 和 abstractive 模型  

[Get To The Point: Summarization with Pointer-Generator Networks](https://arxiv.org/abs/1704.04368)，[code](https://github.com/abisee/pointer-generator)  

把 seq2seq 模型应用于摘要生成时存在两个主要的问题：  

- 难以准确复述原文的事实细节、无法处理原文中的未登录词(OOV)  
- 生成的摘要中存在重复的片段  

针对这两个问题，本文提出融合了 seq2seq 模型和 pointer network 的 pointer-generator network，在CNN/Daily Mail数据集上，相比于state-of-art，ROUGE分数提升了两个点。  

1、seq2seq+attention（baseline）  

seq2seq 模型最早被用于机器翻译任务，由于生成式摘要与翻译任务的相似性，所以 seq2seq 模型也被拿来做生成式摘要。一个 seq2seq 模型通常由三部分组成：encoder、decoder 和 attention 机制。本文中，encoder 部分采用一个单层双向 LSTM，输入原文的词向量序列，输出一个编码后的隐层状态序列 $h_i$；decoder 部分采用一个单层单向 LSTM，每一步的输入是前一步预测的词的词向量，同时输出一个解码的状态序列 $s_t$,用于当前步的预测；attention 是针对原文的概率分布，目的在于告诉模型在当前步的预测过程中，原文中的哪些词更重要。  

2、Pointer-generator network  

pointer-generator network 是 seq2seq 模型和 pointer network 的混合模型，一方面通过 seq2seq 模型保持抽象生成的能力，另一方面通过 pointer network 直接从原文中取词，提高摘要的准确度和缓解 OOV 问题。在预测的每一步，通过动态计算一个生成概率 $p_{gen}$, 把二者软性地结合起来。  

pointer-generator network 相当于在每次摘要生成过程中，都会把原文动态地加入到词表中去，并且在每一步的预测过程中，相比于单纯的 seq2seq 模型，选原文中出现的词作为摘要的概率要更大一些。  

## 3、评价指标  

### 3.1 ROUGE-N  

ROUGE（Recall-Oriented Understudy for Gisting Evaluation），在 2004 年由 ISI 的 Chin-Yew Lin 提出的一种自动摘要评价方法，现被广泛应用于DUC（Document Understanding Conference）的摘要评测任务中。ROUGE 基于摘要中 n 元词(n-gram)的共现信息来评价摘要，是一种面向 n 元词召回率的评价方法。基本思想为由多个专家分别生成人工摘要，构成标准摘要集，将系统生成的自动摘要与人工生成的标准摘要相对比，通过统计二者之间重叠的基本单元(n元语法、词序列和词对)的数目，来评价摘要的质量。通过与专家人工摘要的对比，提高评价系统的稳定性和健壮性。该方法现已成为摘要评价技术的通用标注之一。  

ROUGE 准则由一系列的评价方法组成，包括ROUGE-N(N=1、2、3、4，分别代表基于1元词到4元词的模型)，ROUGE-L，ROUGE-S, ROUGE-W，ROUGE-SU等。在自动文摘相关研究中，一般根据自己的具体研究内容选择合适的 ROUGE 方法。

ROUGE-N 可以通过系统生成摘要与标准摘要相匹配的 N-gram 个数与标准摘要中所有的 N-gram 个数的比值计算得到。  

ROUGE 只计算召回率，而不计算准确率，而且也不管候选摘要流不流畅。  

### 3.2、BLEU  

BLEU 与 ROUGE 计算过程类似，比较并统计系统生成摘要与标准摘要共同出现的 n 元词的个数，即统计同时出现在系统生成摘要和标准摘要中的 n 元词的个数，最后把匹配到的 n 元词的数目除以系统生成摘要的单词数目，得到评测结果。  

BLEU 只关注生成的准确率，所以 BLEU 指标偏向于较短的生成结果。